In [ ]:
# 使用xDeepFM模型对Avazu CTR进行预估
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from deepctr.models import xDeepFM
from deepctr.inputs import SparseFeat,get_feature_names



In [2]:
file_path = './'
fp_train_f = file_path + "avazu_data_10k.csv" #使用小样本进行训练


In [3]:
##==================== xDeepFM 训练 ====================##
data = pd.read_csv(fp_train_f, dtype={'id':str}, index_col=None)
print('data loaded')


data loaded


DeepCTR version 0.7.4 detected. Your version is 0.7.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.7.4


In [4]:
print(data.head())

                       id  click      hour    C1  banner_pos   site_id  \
0  1.0000094181510943e+18      0  14102100  1005           0  1fbe01fe   
1  1.0000169349117864e+19      0  14102100  1005           0  1fbe01fe   
2   1.000037190421512e+19      0  14102100  1005           0  1fbe01fe   
3   1.000064072448084e+19      0  14102100  1005           0  1fbe01fe   
4  1.0000679056417042e+19      0  14102100  1005           1  fe8cc448   

  site_domain site_category    app_id app_domain  ... device_type  \
0    f3845767      28905ebd  ecad2386   7801e8d9  ...           1   
1    f3845767      28905ebd  ecad2386   7801e8d9  ...           1   
2    f3845767      28905ebd  ecad2386   7801e8d9  ...           1   
3    f3845767      28905ebd  ecad2386   7801e8d9  ...           1   
4    9166c161      0569f928  ecad2386   7801e8d9  ...           1   

  device_conn_type    C14  C15  C16   C17  C18  C19     C20  C21  
0                2  15706  320   50  1722    0   35      -1   79  
1     

In [5]:
#数据加载
sparse_features = ['C1', 'banner_pos', 'site_domain', 'site_id','site_category','app_id','app_category', 'device_type', 'device_conn_type','C14', 'C15','C16']
target = ['click']

In [6]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(fixlen_feature_columns)
print(feature_names)

[SparseFeat(name='C1', vocabulary_size=6, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'), SparseFeat(name='banner_pos', vocabulary_size=4, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='banner_pos', group_name='default_group'), SparseFeat(name='site_domain', vocabulary_size=317, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_domain', group_name='default_group'), SparseFeat(name='site_id', vocabulary_size=381, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_id', group_name='default_group'), SparseFeat(name='site_category', vocabulary_size=14, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_category', group_name='default_group'), SparseFeat(name='app_id', vocabulary_size=313, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='app_id', group_name='default_group'), SparseFeat(name='app_category', vocabulary_size=14, embedding_dim=4, use_hash=False,

In [7]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [8]:
# 使用xDeepFM进行训练
model = xDeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
# binary_crossentropy就是logloss
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用xDeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

Train on 6400 samples, validate on 1600 samples


C:\Users\34330\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\34330\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


6400/6400 [==============================] - 12s 2ms/sample - loss: 0.5250 - binary_crossentropy: 0.5250 - val_loss: 0.4710 - val_binary_crossentropy: 0.4710
test RMSE 0.37027017163147236


In [9]:
# 输出LogLoss
from sklearn.metrics import log_loss
score = log_loss(test[target].values, pred_ans)
print("LogLoss", score)


LogLoss 0.4445037735104561


In [11]:
tosub=pd.read_csv("./test.csv", dtype={'id':str}, index_col=None)

In [12]:
tosub_model_input = {name:tosub[name].values for name in feature_names}